<a href="https://colab.research.google.com/github/hector6298/The-Impact-of-Data-Balancing-and-Pre-processing-on-CNNs-Applied-to-CNNs/blob/master/covdIENBF3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive, files
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
!pip3 install pydicom
!pip uninstall -y kaggle
!pip install --upgrade pip
!pip3 install --upgrade kaggle

     |████████████████████████████████| 35.5MB 88kB/s 
Uninstalling kaggle-1.5.6:
  Successfully uninstalled kaggle-1.5.6
     |████████████████████████████████| 1.5MB 7.0MB/s 
  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1
     |████████████████████████████████| 58 kB 3.0 MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.6-py3-none-any.whl size=72859 sha256=eb0b5ec7044e8cfce4d13088c572079c93e81ef6fad6b0ba694695151d58ff26
  Stored in directory: /root/.cache/pip/wheels/01/3e/ff/77407ebac3ef71a79b9166a8382aecf88415a0bcbe3c095a01
Successfully built kaggle


In [ ]:
!mkdir /root/.kaggle
!echo '{"username":"hector6298","key":"724778e3045b27ede8002c9f01b9da72"}' > /root/.kaggle/kaggle.json

In [ ]:
!git clone https://github.com/ieee8023/covid-chestxray-dataset.git
!git clone https://github.com/agchung/Figure1-COVID-chestxray-dataset
!git clone https://github.com/agchung/Actualmed-COVID-chestxray-dataset
!kaggle datasets download -d "tawsifurrahman/covid19-radiography-database"
!kaggle competitions download -c "rsna-pneumonia-detection-challenge" 

Cloning into 'covid-chestxray-dataset'...
remote: Enumerating objects: 3213, done.
remote: Total 3213 (delta 0), reused 0 (delta 0), pack-reused 3213
Receiving objects: 100% (3213/3213), 582.57 MiB | 8.41 MiB/s, done.
Resolving deltas: 100% (1276/1276), done.
Checking out files: 100% (991/991), done.
Cloning into 'Figure1-COVID-chestxray-dataset'...
remote: Enumerating objects: 112, done.
remote: Counting objects: 100% (112/112), done.
remote: Compressing objects: 100% (97/97), done.
remote: Total 112 (delta 28), reused 95 (delta 14), pack-reused 0
Receiving objects: 100% (112/112), 14.13 MiB | 11.32 MiB/s, done.
Resolving deltas: 100% (28/28), done.
Cloning into 'Actualmed-COVID-chestxray-dataset'...
remote: Enumerating objects: 10, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 422 (delta 1), reused 6 (delta 1), pack-reused 412
Receiving objects: 100% (422/422), 1.56 GiB | 22.14 MiB/s, done.
Resolving deltas: 100% (16/

In [ ]:
!unzip rsna-pneumonia-detection-challenge.zip
!rm rsna-pneumonia-detection-challenge.zip
!unzip covid19-radiography-database.zip
!rm covid19-radiography-database.zip
!mkdir data
!mkdir data/train
!mkdir data/test
!mkdir /content/logs

In [ ]:
!python3 "/content/drive/My Drive/COVID-Net-master/COVID-Net-master/create_covidx_v3.py"

Data distribution from covid datasets:
{'normal': 0, 'pneumonia': 40, 'COVID-19': 568}
Key:  pneumonia
Test patients:  ['8', '31']
Key:  COVID-19
Test patients:  ['19', '20', '36', '42', '86', '94', '97', '117', '132', '138', '144', '150', '163', '169', '174', '175', '179', '190', '191COVID-00024', 'COVID-00025', 'COVID-00026', 'COVID-00027', 'COVID-00029', 'COVID-00030', 'COVID-00032', 'COVID-00033', 'COVID-00035', 'COVID-00036', 'COVID-00037', 'COVID-00038', 'ANON24', 'ANON45', 'ANON126', 'ANON106', 'ANON67', 'ANON153', 'ANON135', 'ANON44', 'ANON29', 'ANON201', 'ANON191', 'ANON234', 'ANON110', 'ANON112', 'ANON73', 'ANON220', 'ANON189', 'ANON30', 'ANON53', 'ANON46', 'ANON218', 'ANON240', 'ANON100', 'ANON237', 'ANON158', 'ANON174', 'ANON19', 'ANON195', 'COVID-19(119)', 'COVID-19(87)', 'COVID-19(70)', 'COVID-19(94)', 'COVID-19(215)', 'COVID-19(77)', 'COVID-19(213)', 'COVID-19(81)', 'COVID-19(216)', 'COVID-19(72)', 'COVID-19(106)', 'COVID-19(131)', 'COVID-19(107)', 'COVID-19(116)', 'COVI

In [ ]:
import tensorflow as tf
from tensorflow import keras

import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
from skimage.exposure import equalize_adapthist
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.utils import shuffle
from sklearn.metrics import average_precision_score
from tensorflow.python.keras.callbacks import Callback
from sklearn.metrics import recall_score, classification_report
from tensorflow.keras.callbacks import ReduceLROnPlateau
from sklearn.utils import class_weight

In [ ]:
def _process_csv_file(file):
    with open(file, 'r') as fr:
        files = fr.readlines()
    return files

In [ ]:
def xray_enhance(img):
  gaussian_3 = cv2.GaussianBlur(img, (3,3), 10.0)
  unsharp_image = cv2.addWeighted(img, 1.5, gaussian_3, -0.5, 0, img)
  return unsharp_image

In [ ]:
from itertools import compress
def clean_annotation(annotations, train_dir, test_dir):
  files = []
  for r, d, f in os.walk(train_dir):
    for file in f:
      #print(os.path.join(r, file))
      files.append(os.path.join(r, file))
  for r, d, f in os.walk(test_dir):
    for file in f:
      #print(os.path.join(r, file))
      files.append(os.path.join(r, file))

  mask = np.ones(len(annotations), dtype=bool)

  for i in range(len(annotations)):
    if f"{annotations[i][1]}" not in files:
      print(f"Deleting {annotations[i][1]}")
      mask[i] = False
  return list(compress(annotations, mask)), files

In [ ]:
EPOCHS = 20
LEARNING_RATE = 0.001
BATCH_SIZE = 64
INPUT_SIZE = 224
NCLASSES = 3
INPUT_SHAPE = (INPUT_SIZE, INPUT_SIZE, NCLASSES)
ENHANCE = True
TRAINFILE = '/content/train_split_v3.txt'
TESTFILE = '/content/test_split_v3.txt'
DATADIR = '/content/data'



In [ ]:
#dataset for fold 1
dataset_train = _process_csv_file(TRAINFILE)
dataset_test = _process_csv_file(TESTFILE)
datasets = {'normal': [], 'pneumonia': [], 'COVID-19': []}
for l in dataset_train:
  entry = l.split()
  entry[1] = f"data/train/{entry[1]}"
  datasets[entry[2]].append(entry)
for l in dataset_test:
  entry = l.split()
  entry[1] = f"data/test/{entry[1]}"
  datasets[entry[2]].append(entry)

break_point_normal = int(len(datasets['normal'])/5)
break_point_covid = int(len(datasets['COVID-19'])/5)
break_point_pneumonia = int(len(datasets['pneumonia'])/5)



In [ ]:
train_set = [datasets['normal'][0:(break_point_normal*4)] + datasets['pneumonia'][0:(break_point_pneumonia*4)] + datasets['COVID-19'][0:(break_point_covid*4)]]
test_set = [datasets['normal'][(break_point_normal*4):] + datasets['pneumonia'][(break_point_pneumonia*4):] + datasets['COVID-19'][(break_point_covid*4):]]

In [ ]:
train_set1 = datasets['normal'][0:(break_point_normal*3)] + datasets['pneumonia'][0:(break_point_pneumonia*3)] + datasets['COVID-19'][0:(break_point_covid*3)]
train_set2 = datasets['normal'][(break_point_normal*4):] + datasets['pneumonia'][(break_point_pneumonia*4):] + datasets['COVID-19'][(break_point_covid*4):]
train_set = [train_set1 + train_set2]
test_set = [datasets['normal'][(break_point_normal*3):(break_point_normal*4)] + datasets['pneumonia'][(break_point_pneumonia*3):(break_point_pneumonia*4)] + datasets['COVID-19'][(break_point_covid*3):(break_point_covid*4)]]

In [ ]:
train_set = [datasets['normal'][(break_point_normal*1):] + datasets['pneumonia'][(break_point_pneumonia*1):] + datasets['COVID-19'][(break_point_covid*1):]]
test_set = [datasets['normal'][0:(break_point_normal*1)] + datasets['pneumonia'][0:(break_point_pneumonia*1)] + datasets['COVID-19'][0:(break_point_covid*1)]]

In [ ]:
print(len(test_set[0]))

3093


In [ ]:
print("go for train...")
annotations_train, filestr = clean_annotation(train_set[0], "data/train", "data/test")
print("go for test...")
annotations_test, fileste = clean_annotation(test_set[0], "data/train", "data/test")
print('done!')

go for train...
go for test...
done!


In [ ]:
print(len(annotations_train))

12378


In [ ]:
from tensorflow import keras

import numpy as np
import os
import cv2

from tensorflow.keras.preprocessing.image import ImageDataGenerator

def crop_top(img, percent=0.15):
    offset = int(img.shape[0] * percent)
    return img[offset:]

def central_crop(img):
    size = min(img.shape[0], img.shape[1])
    offset_h = int((img.shape[0] - size) / 2)
    offset_w = int((img.shape[1] - size) / 2)
    return img[offset_h:offset_h + size, offset_w:offset_w + size]

def process_image_file(filepath, top_percent, size, enhance=False):
    img = cv2.imread(filepath)
    img = crop_top(img, percent=top_percent)
    img = central_crop(img)
    img = cv2.resize(img, (size, size))
    if enhance:
      img = xray_enhance(img)
    return img

_augmentation_transform = ImageDataGenerator(
    featurewise_center=False,
    featurewise_std_normalization=False,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    brightness_range=(0.9, 1.1),
    zoom_range=(0.85, 1.15),
    fill_mode='constant',
    cval=0.,
)

def apply_augmentation(img):
    img = _augmentation_transform.random_transform(img)
    return img

class BalanceCovidDataset(keras.utils.Sequence):
    'Generates data for Keras'

    def __init__(
            self,
            dataset,
            is_training=True,
            batch_size=8,
            input_shape=(224, 224),
            n_classes=3,
            num_channels=3,
            mapping={
                'normal': 0,
                'pneumonia': 1,
                'COVID-19': 2
            },
            shuffle=True,
            augmentation=apply_augmentation,
            covid_percent=0.3,
            top_percent=0.08,
            balancing=False,
            enhance=False,
            aug=False
    ):
        'Initialization'
        self.dataset = dataset
        self.is_training = is_training
        self.batch_size = batch_size
        self.N = len(self.dataset)
        self.input_shape = input_shape
        self.n_classes = n_classes
        self.num_channels = num_channels
        self.mapping = mapping
        self.shuffle = True
        self.covid_percent = covid_percent
        self.n = 0
        self.augmentation = augmentation
        self.top_percent = top_percent
        self.balancing = balancing
        self.mean = None
        self.std = None
        self.enhance = enhance
        self.aug=aug
        datasets = {'normal': [], 'pneumonia': [], 'COVID-19': []}
        for l in self.dataset:
          datasets[l[2]].append(l)
        if self.balancing:
          self.datasets = [
              datasets['normal'] + datasets['pneumonia'],
              datasets['COVID-19'],
          ]
        else:
          self.datasets = [
              datasets['normal'] + datasets['pneumonia'] + datasets['COVID-19']
          ]
          #print(len(self.datasets[0]), len(self.datasets[1]))

        self.on_epoch_end()

    def __next__(self):
        # Get one batch of data
        batch_x, batch_y = self.__getitem__(self.n)
        # Batch index
        self.n += 1

        # If we have processed the entire dataset then
        if self.n >= self.__len__():
            self.on_epoch_end
            self.n = 0

        return batch_x, batch_y

    def __len__(self):
        return int(np.ceil(len(self.datasets[0]) / float(self.batch_size)))

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        if self.shuffle == True:
            for v in self.datasets:
                np.random.shuffle(v)

    def include_statistics(mean, std):
      self.mean = mean
      self.std = std

    def __getitem__(self, idx):
        batch_x, batch_y = np.zeros(
            (self.batch_size, *self.input_shape,
             self.num_channels)), np.zeros(self.batch_size)

        batch_files = self.datasets[0][idx * self.batch_size:(idx + 1) *
                                       self.batch_size]
        if self.balancing:
       # upsample covid cases
          covid_size = max(int(len(batch_files) * self.covid_percent), 1)
          covid_inds = np.random.choice(np.arange(len(batch_files)),
                                        size=covid_size,
                                        replace=False)
         
          choices_indices = np.random.choice(np.arange(len(self.datasets[1])),
                                        size=covid_size,
                                        replace=False)
          covid_files = [self.datasets[1][i] for i in choices_indices]
          for i in range(covid_size):
              batch_files[covid_inds[i]] = covid_files[i]

        for i in range(len(batch_files)):
            sample = batch_files[i]


            x = process_image_file(sample[1],
                                   self.top_percent,
                                   self.input_shape[0], self.enhance)

            if self.is_training and hasattr(self, 'augmentation') and self.aug:
                x = self.augmentation(x)

            x = x.astype('float32') / 255.0
            if self.mean is not None and self.std is not None:
              x = (x-mean)/std

            y = self.mapping[sample[2]]

            batch_x[i] = x
            batch_y[i] = y

        return batch_x, keras.utils.to_categorical(batch_y, num_classes=self.n_classes)

In [ ]:
train_generator = BalanceCovidDataset(
                                dataset=annotations_train,
                                batch_size=BATCH_SIZE,
                                input_shape=(INPUT_SIZE, INPUT_SIZE),
                                top_percent=0.08,
                                balancing=False,
                                enhance=ENHANCE,
                                aug=False)

test_generator = BalanceCovidDataset(
                                dataset=annotations_test,
                                batch_size=BATCH_SIZE,
                                is_training=False,
                                input_shape=(INPUT_SIZE, INPUT_SIZE),
                                top_percent=0.08,
                                balancing=False,
                                enhance=ENHANCE,
                                aug=False)
                                


In [ ]:
base_model = tf.keras.applications.Xception(
    include_top=False, weights='imagenet', input_shape=INPUT_SHAPE,
)
base_model.trainable = True



83689472/83683744 [==============================] - 4s 0us/step


In [ ]:
def get_model(base_model, input_shape):
  inputs = tf.keras.layers.Input(shape=input_shape)
  global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
  prediction_layer = tf.keras.layers.Dense(NCLASSES, activation='softmax')

  x = inputs
  x = base_model(x)
  x = global_average_layer(x)
  x = prediction_layer(x)

  return tf.keras.Model(inputs=inputs, outputs=x)

In [ ]:
METRICS = [
  tf.keras.metrics.CategoricalAccuracy(),

  keras.metrics.Precision(name='precision_0', class_id=0),
  keras.metrics.Recall(name='recall_0', class_id=0),
  keras.metrics.AUC(name='auc_all', multi_label=True),

  keras.metrics.Precision(name='precision_1', class_id=1),
  keras.metrics.Recall(name='recall_1', class_id=1),

  keras.metrics.Precision(name='precision_2', class_id=2),
  keras.metrics.Recall(name='recall_2', class_id=2),
]


In [ ]:
metrics = METRICS
optimizer = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE)
loss_object = tf.keras.losses.CategoricalCrossentropy()

model = get_model(base_model, INPUT_SHAPE)

model.compile(optimizer=optimizer,
              loss=loss_object,
              metrics = metrics)

In [ ]:
def scheduler(epoch):
  if epoch <= 5:
    return 0.005
  elif epoch > 5 and epoch <= 10:
    return 0.001
  else:
    return 0.001 * tf.math.exp(0.1 * (10 - epoch))
scheduler_callback = tf.keras.callbacks.LearningRateScheduler(scheduler)
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="/content/logs")

In [ ]:
model_history = model.fit(x=train_generator, 
                epochs=EPOCHS,
                validation_data=test_generator,
                verbose=1,
                callbacks=[scheduler_callback, tensorboard_callback]
                )

Epoch 1/20
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
194/194 [==============================] - 302s 2s/step - loss: 0.3650 - categorical_accuracy: 0.8672 - precision_0: 0.9053 - recall_0: 0.8954 - auc_all: 0.9302 - precision_1: 0.8800 - recall_1: 0.8015 - precision_2: 0.8146 - recall_2: 0.2703 - val_loss: 1.3077 - val_categorical_accuracy: 0.8017 - val_precision_0: 0.8935 - val_recall_0: 0.7959 - val_auc_all: 0.7625 - val_precision_1: 0.7041 - val_recall_1: 0.8851 - val_precision_2: 0.0000e+00 - val_recall_2: 0.0000e+00 - lr: 0.0050
Epoch 2/20
194/194 [==============================] - 295s 2s/step - loss: 0.2293 - categorical_accuracy: 0.9175 - precision_0: 0.9299 - recall_0: 0.9463 - auc_all: 0.9720 - precision_1: 0.9218 - recall_1: 0.8860 - precision_2: 0.8793 - recall_2: 0.6725 - val_loss: 1.6119 - val_categorical_accuracy: 0.7905 - val_precision_0: 0.7771 - val_recall_0: 0.9923 - val_auc_all: 0.8724 - val_precision_1: 0.9815 - val_recall_1: 

In [ ]:
@tf.function
def test_step(test_images, test_labels):
  return model(test_images)

In [ ]:
y_pred = []
y_true = []

for j in range(len(test_generator)):

    test_images, test_labels = next(test_generator)
    #print(test_labels)
    
    #imgs += len(test_images)
    predictions = test_step(test_images, test_labels)
    predictions_classnum = np.argmax(predictions, axis=1)
    test_labels_ = np.argmax(test_labels, axis=1)
    confidences = np.amax(predictions)

    for item in range(BATCH_SIZE):
        y_pred.append(predictions_classnum[item])
        y_true.append(int(test_labels_[item]))

In [ ]:
target_names = ['Normal', 'Pneumonia', 'COVID-19']
print(classification_report(y_true, y_pred, target_names=target_names))

              precision    recall  f1-score   support

      Normal       0.93      0.96      0.95      1813
   Pneumonia       0.93      0.87      0.90      1210
    COVID-19       0.72      0.79      0.75       113

    accuracy                           0.92      3136
   macro avg       0.86      0.87      0.87      3136
weighted avg       0.92      0.92      0.92      3136



In [ ]:

model.save_weights('covid_net_weights_IENBF5.h5')
files.download('covid_net_weights_IENBF5.h5')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!zip -r /content/logsRB.zip /content/logs

In [ ]:
files.download("/content/logsRB.zip")